## ex10-Using Views to Simplify Queries


One of the beautiful aspects of the relational data model and SQL is that the output of a query is also a table, a relation to be precise. It may consist of a single column or a single row, but it is a table nonetheless. A view is a query that can be used like a table. A view can be considered as a virtual table that does not hold data. They just hold a query. Every time a view is accessed, the query underlying it is run and the returned results can be used as though they made up an actual table.

There are [several reasons](http://www.sqlitetutorial.net/sqlite-create-view/) for using veiws. I think the utmost reason is we are lazy and we do not want to write the same complicated long query sentences every time. :) I am kidding. However, keep in mind the DRY programming principle: Don’t Repeat Yourself. Avoiding repetition saves time and prevents unnecessary mistakes. This is one of right reasons that we save queries as reusable database views.

SQLite views are created using the CREATE VIEW statement. SQLite views can be created from a single table, multiple tables, or another view. Following is the basic [CREATE VIEW syntax](http://www.sqlitetutorial.net/sqlite-create-view/):
>CREATE [TEMP | TEMPORARY] VIEW view_name AS<br>
>SELECT column1, column2.....<br>
>FROM table_name<br>
>WHERE [condition];

SQLite view is read only. It means you cannot use INSERT, DELETE, and UPDATE statement to update data in the base tables through the view.

In [1]:
%load_ext sql

### 1. Connet to the given database of demo.db3

In [2]:
%sql sqlite:///data/demo.db3

u'Connected: @data/demo.db3'

If you do not remember the tables in the demo data, you can always use the following command to query.

In [3]:
%sql SELECT name FROM sqlite_master WHERE type='table'

 * sqlite:///data/demo.db3
Done.


name
rch
hru
sub
sed
watershed_daily
watershed_monthly
watershed_yearly
channel_dimension
hru_info
sub_info


### 2. Simplifying queries with views

In the previous notebook, we used CASE and Subquery to calculate seasonal runoff from the table of rch. Here we use a view to simplify the calculation.

#### 2.1 Have a recall of how to calculate season runoffs 

In [4]:
%%sql sqlite://
SELECT RCH, Quarter, AVG(FLOW_OUTcms) as Runoff
FROM(
SELECT RCH, YR, 
CASE 
    WHEN (MO) BETWEEN 3 AND 5 THEN 'MAM'   
    WHEN (MO) BETWEEN 6 and 8 THEN 'JJA'
    WHEN (MO) BETWEEN 9 and 11 THEN 'SON'
    ELSE 'DJF' 
END Quarter,
FLOW_OUTcms
from rch)
GROUP BY RCH, Quarter
LIMIT 5

Done.


RCH,Quarter,Runoff
1,DJF,99.2049905353
1,JJA,1405.26229799
1,MAM,559.746932019
1,SON,454.737985314
2,DJF,56.3285390854


#### 2.2 Creating a view

In [5]:
%%sql sqlite://
CREATE VIEW RCH_VW AS SELECT RCH, YR, 
CASE 
    WHEN (MO) BETWEEN 3 AND 5 THEN 'MAM'   
    WHEN (MO) BETWEEN 6 and 8 THEN 'JJA'
    WHEN (MO) BETWEEN 9 and 11 THEN 'SON'
    ELSE 'DJF' 
END Quarter,
FLOW_OUTcms
from rch

Done.


[]

Let's query the SSN_RCH view

In [6]:
%%sql sqlite://
SELECT * 
FROM RCH_VW 
LIMIT 5

Done.


RCH,YR,Quarter,FLOW_OUTcms
1,1981,DJF,146.252487183
2,1981,DJF,96.1828536987
3,1981,DJF,11.8613681793
4,1981,DJF,49.4065132141
5,1981,DJF,272.106018066


### 2.3 Recalculate seasonal runoffs with views

:) The codes really really get shorter.

In [7]:
%%sql sqlite://
SELECT RCH, Quarter, AVG(FLOW_OUTcms) as Runoff
FROM RCH_VW
GROUP BY RCH, Quarter
LIMIT 5

Done.


RCH,Quarter,Runoff
1,DJF,99.2049905353
1,JJA,1405.26229799
1,MAM,559.746932019
1,SON,454.737985314
2,DJF,56.3285390854


### 2.4 Deleting Views

It is quite easy to delete views. Just drop it like the following.

In [8]:
%sql DROP VIEW RCH_VW

 * sqlite:///data/demo.db3
Done.


[]

### Summary

Views are virtual tables that do not hold data, only SQL statements. Those statements are executed each time the view is accessed. Because views are created dynamically as they are accessed and the data in those views are always fresh and up-to-date, they have some advantages over creating a subtables from a table. The data in subtables is static and could be out-to-date.

A view is useful in some [cases](http://www.sqlitetutorial.net/sqlite-create-view/):
- First, views provide an abstraction layer over tables. You can add and remove the columns in the view without touching the schema of the underlying tables.
- Second, you can use views to encapsulate complex queries with joins to simplify the data access.